# 0. Import the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import scipy.optimize
import statsmodels.stats
import scipy.stats

EPS = np.finfo(np.float).eps

# 1. Define your experiment: Task model

## Bandit task

Parameters: 
* Number of option ($N$)
* Distribution of probability over these options ($\{p_{reward}(i)\}_{i\in N}$)
* Number of trials ($T$)

In [ ]:
N = 2
P = np.array([0.5, 0.75])
T = 100

# 2. Define your decision-making models: Cognitive model

## Random

* Decision rule ($\tau$)

\begin{equation}
p_{choice}(i) \sim \text{Uniform}
\end{equation} 

* Updating rule

[None]

## Rescola-Wagner RL

* Decision rule ($\tau$)

\begin{equation}
p_{choice}(i) = \dfrac{\exp (v(i)/\tau)}{\sum_{j \in N} \exp (v(j)/\tau)} 
\end{equation}

* Updating rule ($\alpha$)

\begin{equation}
v^{t+1}(i) = v^t(i) + \alpha(s - v^t(i))
\end{equation}

In [ ]:
class Random:
    
    """
    No-learning model
    """
    
    def __init__(self, n_option):
        self.n_option = n_option
        self.options = np.arange(n_option)
        
    def choose(self):
        p =  self.decision_rule()
        return np.random.choice(self.options, p=p)
        
    def learn(self, i, success):
        self.updating_rule(i, success)
    
    def decision_rule(self):
        return np.ones(self.n_option) / self.n_option
    
    def updating_rule(self, i, success):
        pass
        

In [ ]:
class RL(Random):
    
    """
    Reinforcement learning model
    """
    
    def __init__(self, learning_rate, temp, n_option, initial_value=0.5):
        super().__init__(n_option=n_option)
        self.values = np.full(n_option, initial_value)
        self.learning_rate = learning_rate
        self.temp = temp
    
    def decision_rule(self):
        return np.exp(self.values/self.temp) / \
               np.sum(np.exp(self.values/self.temp))
        
    def updating_rule(self, i, success):
        self.values[i] += self.learning_rate * (success - self.values[i])

# 3. Simulate 

In [ ]:
def run_simulation(agent_model, param, n_iteration, n_option,
                   prob_dist):
    
    if param is not None:
        agent = agent_model(n_option=n_option, *param)
    else:
        agent = agent_model(n_option=n_option)
    
    choices = np.zeros(n_iteration, dtype=int)
    successes = np.zeros(n_iteration, dtype=bool)
    
    # Simulate the task
    for t in range(n_iteration):
        
        # Determine choice
        choice = agent.choose()
        
        # Determine success
        p_success = prob_dist[choice]
        success = np.random.choice([0, 1], p=np.array([1-p_success, p_success]))
        
        # Make agent learn
        agent.learn(i=choice, success=success)
        
        # Backup
        choices[t] = choice
        successes[t] = success
    
    return choices, successes

In [ ]:
np.random.seed(0)

learning_rate, temp = 0.01, 0.1

models = Random, RL
params = None, (learning_rate, temp)

n_models = len(models)

hist_choices = {}
hist_successes = {}

# Simulate the task
for i in range(n_models):
    
    m = models[i]
    
    choices, successes = run_simulation(
        agent_model=m, param=params[i],
        n_iteration=T, n_option=N, prob_dist=P)
    
    hist_choices[m.__name__] = choices
    hist_successes[m.__name__] = successes

In [ ]:
def basic_scatter(data, y_label="choice", x_label="time"):

    keys = sorted(data.keys())
    n_keys = len(keys)

    fig, axes = plt.subplots(ncols=n_keys, figsize=(10, 4))

    colors = [f'C{i}' for i in range(n_keys)]

    for i in range(n_keys):

        k = keys[i]
        y = data[k]

        ax = axes[i]
        ax.scatter(range(len(y)), y, color=colors[i],
                   alpha=0.2, label=k)

        ax.set_ylim(-0.02, 1.02)
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.legend()

    plt.tight_layout()
    plt.show()

basic_scatter(data=hist_choices)

In [ ]:
def running_mean(data, y_label='choice', x_label="time", window=20):

    keys = sorted(data.keys())
    n_keys = len(keys)

    fig, axes = plt.subplots(ncols=n_keys)

    colors = [f'C{i}' for i in range(n_keys)]

    for i in range(n_keys):

        k = keys[i]

        y = data[k]

        ax = axes[i]
        ax.plot(pd.Series(y).rolling(window).mean(),
                color=colors[i], alpha=0.2, label=k)
        ax.set_ylim(-0.02, 1.02)

        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)

        ax.legend()
    plt.tight_layout()
    plt.show()

running_mean(data=hist_choices)

In [ ]:
import statsmodels.stats.proportion 
import scipy.stats
for i in range(n_models):
    
    m = models[i]
    choices = hist_choices[m.__name__]
    
    k, n = np.sum(choices), len(choices)
    print(statsmodels.stats.proportion.proportion_confint(count=k, nobs=n))
    freq_yes = k/n
    freq_no = 1- freq_yes
    chi2, p = scipy.stats.chisquare((k, n-k))
    print(chi2, p)

In [ ]:
class BanditOptimizer:
    
    def __init__(self,
                 n_option, 
                 choices,
                 successes,
                 model, bounds=None):
        
        self.n_option = n_option
        self.choices = choices
        self.successes = successes
        self.model = model
        self.bounds = bounds

    def objective(self, param=None):
    
        n_iteration = len(choices)
        agent = self.model(n_option=self.n_option, *param)
    
        log_likelihood = np.zeros(n_iteration)
    
       # Simulate the task
        for t in range(n_iteration):
            
            choice, success = choices[t], successes[t]
            
            ps = agent.decision_rule()
            p_choice = ps[choice]
            
            log_likelihood[t] = np.log(p_choice + EPS)
            
            # Make agent learn
            agent.learn(i=choice, success=success)

        return np.sum(log_likelihood)
    
    def _func(self, param):
        return self.objective(param)
        
    def run(self):

        res = scipy.optimize.differential_evolution(
            func=self._func,
            bounds=self.bounds)
        best_param = res.x
        best_value = res.fun
        
        return best_param, best_value


Determine le (log) likelihood of your model

\begin{equation}
\log p(D \mid M, \theta) = \sum_{d \in D} \log p(d \mid M, \theta)
\end{equation}



Parameter recovery

In [ ]:
param_labels = "alpha", "beta"
bounds = (0, 1), (0.01, 1)

n_sets = 30

n_param = len(param_labels)

param = np.zeros((n_sets, n_param))
param_recovered = np.zeros(param.shape)

for i in range(n_sets):
    
    param = np.zeros(n_param)
    for j in range(n_param):
        
        param[j] = np.random.uniform(*bounds[j])
        
    choices, successes = \
        run_simulation(
            agent_model=RL, 
            param=param, 
            n_iteration=T, 
            n_option=N,
            prob_dist=P
        )
    
    opt = BanditOptimizer(
        n_option=N,
        choices=choices,
        successes=successes,
        model=RL,
        bounds=bounds
    )
    best_param, best_value = opt.run()
    print("best param", best_param)
    print("best value", best_value)

In [ ]:
def scatter(data):
    # Create fig
    fig, axes = plt.subplots(nrows=len(data.keys()), figsize=(5, 10))

    i = 0
    for title, array_like in sorted(data.items()):

        ax = axes[i]

        x, y = array_like

        ax.scatter(x, y, alpha=0.5)

        ax.set_title(title)

        max_ = max(x+y)
        min_ = min(x+y)

        ax.set_xlim(min_, max_)
        ax.set_ylim(min_, max_)

        ticks_positions = [round(i, 2) for i in np.linspace(min_, max_, 3)]

        ax.set_xticks(ticks_positions)
        ax.set_yticks(ticks_positions)

        ax.set_aspect(1)
        i += 1

def correlation(x, y):
    cor, p = scipy.stats.pearsonr(x, y)
    return cor, p


\begin{equation}
\text{BIC} = - 2 \log p(D\mid {\widehat {\theta }},M) + k \log(T)
\end{equation}

In [ ]:
def bic(ll, k, n_iteration):
    return -2 *ll + k * np.log(n_iteration)